In [ ]:
import os
import zipfile
import joblib
import yaml
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from collections import OrderedDict
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchsummary import summary
from torchview import draw_graph

In [ ]:
def params():
    with open("../../default_params.yml", "r") as file:
        return yaml.safe_load(file)

In [ ]:
def dump(value = None, filename = None):
    if (value is not None):
        joblib.dump(value = value, filename = filename)
        
    else:
        raise ValueError("The value is empty. Please check the value and try again.")
    
    
def load(filename):
    if filename is not None:
        return joblib.load(filename = filename)
    
    else:
        raise ValueError("The filename is empty. Please check the filename and try again.")

In [ ]:
class Loader:
    def __init__(self, image_path = None, image_size = 128, batch_size = 1, split_size = 0.20):
        self.image_path = image_path
        self.image_size = image_size
        self.batch_size = batch_size
        self.split_size = split_size

        self.sharp_images = []
        self.blurred_images = []

        try:
            self.config = params()

        except Exception as e:
            print("The config file is not valid. Please check the file and try again.".capitalize())

    def unzip_folder(self):
        if os.path.exists(self.config["path"]["raw_path"]):
            with zipfile.ZipFile(self.image_path, "r") as zip_ref:
                zip_ref.extractall(os.path.join(self.config["path"]["raw_path"], "dataset"))

        else:
            raise FileNotFoundError("The raw data folder does not exist. Please check the path and try again.")
        
    
    def split_images(self, **kwargs):
        X_train, X_test, y_train, y_test = train_test_split(
            kwargs["X"], kwargs["y"], test_size=self.split_size, random_state=42
            )
        
        return {"X_train": X_train, "X_test": X_test, "y_train": y_train, "y_test": y_test}

    def transforms(self):
        return transforms.Compose([
            transforms.Resize((self.image_size, self.image_size)),
            transforms.ToTensor(),
            transforms.CenterCrop((self.image_size, self.image_size)),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

    def feature_extractor(self):
        if os.path.exists(self.config["path"]["raw_path"]):
            self.directory = os.path.join(self.config["path"]["raw_path"], "dataset")
            self.sharp = ["sharp"]

            blurred_image = os.path.join(self.directory, "blurred")

            for category in self.sharp:
                path = os.path.join(self.directory, category)

                for image in os.listdir(path):
                    sharp_image_number = image.split("_")[0]

                    for blur_image in os.listdir(blurred_image):
                        blurred_image_number = blur_image.split("_")[0]

                        if sharp_image_number == blurred_image_number:
                            sharp_image = cv2.imread(os.path.join(path, image))
                            blur_image = cv2.imread(os.path.join(blurred_image, blur_image))
                            
                            sharp_image = cv2.cvtColor(sharp_image, cv2.COLOR_BGR2RGB)
                            blur_image = cv2.cvtColor(blur_image, cv2.COLOR_BGR2RGB)

                            self.sharp_images.append(self.transforms()(Image.fromarray(sharp_image)))
                            self.blurred_images.append(self.transforms()(Image.fromarray(blur_image)))
                            
                        else:
                            continue
                            
            dataset = self.split_images(X = self.blurred_images, y = self.sharp_images)

            return {"data": dataset, "sharp": self.sharp_images, "blurred": self.blurred_images}

        else:
            raise FileNotFoundError("The raw data folder does not exist. Please check the path and try again.")

    def create_dataloader(self):
        if os.path.exists(self.config["path"]["processed_path"]):
            
            data = self.feature_extractor()
            
            train_dataloader = DataLoader(
                dataset=list(zip(data["data"]["X_train"], data["data"]["y_train"])),
                batch_size=self.batch_size, shuffle=True
                )
            
            test_dataloader = DataLoader(
                dataset=list(zip(data["data"]["X_test"], data["data"]["y_test"])),
                batch_size=self.batch_size*16, shuffle=True
            )
            
            dataloader = DataLoader(
                dataset=list(zip(data["sharp"], data["blurred"])),
                batch_size=self.batch_size, shuffle=True
                )
            
            dump(
                value=dataloader, filename=os.path.join(self.config["path"]["processed_path"], "dataloader.pkl"))
            
            dump(
                value=train_dataloader, filename=os.path.join(self.config["path"]["processed_path"], "train_dataloader.pkl"))
            
            dump(
                value=test_dataloader, filename=os.path.join(self.config["path"]["processed_path"], "test_dataloader.pkl"))
            
        else:
            raise FileNotFoundError("The processed data folder does not exist. Please check the path and try again.")
        
    
    @staticmethod
    def plot_images():
        config = params()
        
        if os.path.exists(config["path"]["processed_path"]):
            plt.figure(figsize=(40, 15))
            
            test_dataloader = load(
                filename=os.path.join(config["path"]["processed_path"], "test_dataloader.pkl")
                )
            
            blurred, sharp = next(iter(test_dataloader))
            
            for index, image in enumerate(sharp):
                sharp_image = image.squeeze().permute(1, 2, 0).cpu().detach().numpy()
                blurred_image = blurred[index].squeeze().permute(1, 2, 0).cpu().detach().numpy()
                
                sharp_image = (sharp_image - sharp_image.min())/(sharp_image.max() - sharp_image.min())
                blurred_image = (blurred_image - blurred_image.min())/(blurred_image.max() - blurred_image.min())
                
                plt.subplot(2 * 4, 2 * 4, 2 * index + 1)
                plt.imshow(sharp_image)
                plt.title("Sharp")
                plt.axis("off")
                
                plt.subplot(2 * 4, 2 * 4, 2 * index + 2)
                plt.imshow(blurred_image)
                plt.title("Blurred")
                plt.axis("off")
                
            plt.tight_layout()
            
            if os.path.exists(config["path"]["file_path"]):
                plt.savefig(os.path.join(config["path"]["file_path"], "images.jpg"))
                
            plt.show()
            
        else:
            raise FileNotFoundError("The processed data folder does not exist. Please check the path and try again.")
        
        
    
    @staticmethod
    def dataset_details():
        config = params()
        
        if os.path.exists(config["path"]["processed_path"]):
            
            dataloader = load(
                filename=os.path.join(config["path"]["processed_path"], "dataloader.pkl")
                )
            
            test_dataloader = load(
                filename=os.path.join(config["path"]["processed_path"], "test_dataloader.pkl")
                )
            
            train_dataloader = load(
                filename=os.path.join(config["path"]["processed_path"], "train_dataloader.pkl")
                )
            
            train_data, train_label = next(iter(train_dataloader))
            test_data, test_label = next(iter(test_dataloader))
            
            pd.DataFrame(
                {
                    "total_data": str(sum(sharp.size(0) for sharp, _ in dataloader)),
                    "total_train_data": str(sum(sharp.size(0) for sharp, _ in train_dataloader)),
                    "total_test_data": str(sum(sharp.size(0) for sharp, _ in test_dataloader)),
                    "train_data_shape": str(train_data.size()),
                    "train_label_shape": str(train_label.size()),
                    "test_data_shape": str(test_data.size()),
                    "test_label_shape": str(test_label.size())
                },
                index=["quantity".title()]
            ).T.to_csv(os.path.join(config["path"]["file_path"], "dataset_details.csv"))
        else:
            raise FileNotFoundError("The processed data folder does not exist. Please check the path and try again.")

if __name__ == "__main__":
    loader = Loader(
        image_path="/Users/shahmuhammadraditrahman/Desktop/dataset.zip",
        image_size=128,
        split_size=0.20)
    
    loader.unzip_folder()
    
    loader.create_dataloader()
    
    loader.plot_images()
    
    print(loader.dataset_details())
    

### Create the Encoder

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, in_channels = 3, out_channels = 64):
        super(EncoderBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel = 3
        self.stride = 1
        self.padding = 1
        
        self.layers = OrderedDict()
        self.encoder = self.block()
        
    def block(self):
        self.layers["conv"] = nn.Conv2d(
            in_channels=self.in_channels, out_channels=self.out_channels, kernel_size=self.kernel, stride=self.stride, padding=self.padding)
        
        self.layers["relu"] = nn.ReLU(inplace=True)
        
        self.layers["conv_1"] = nn.Conv2d(
            in_channels=self.out_channels, out_channels=self.out_channels, kernel_size=self.kernel, stride=self.stride, padding=self.padding)
        
        self.layers["batch_norm"] = nn.BatchNorm2d(
            num_features=self.out_channels)
        
        self.layers["relu_1"] = nn.ReLU(inplace=True)
        
        self.layers["max_pool"] = nn.MaxPool2d(kernel_size=(self.kernel//self.kernel)*2, stride=self.stride*2)
        
        return nn.Sequential(self.layers)
    
    def forward(self, x):
        if x is not None:
            return self.encoder(x)
        
        else:
            raise ValueError("The input must not be None.".capitalize())
        
    @staticmethod
    def total_params(model = None):
        if model is not None:
            return sum(params.numel() for params in model.parameters())
        
        else:
            raise ValueError("The model must not be None.".capitalize())


if __name__ == "__main__":

    in_channels = 3
    out_channels = 64
    num_repetitive = 5

    layers = []

    for _ in range(num_repetitive):
        layers.append(EncoderBlock(in_channels=in_channels, out_channels=out_channels))

        in_channels = out_channels
        out_channels *= 2

    model = nn.Sequential(*layers)

    print(EncoderBlock.total_params(model=model))

    print(summary(model=model, input_size=(3, 128, 128)))
    
    config = params()

    draw_graph(model=model, input_size=(1, 3, 256, 256)).visual_graph.render(
        filename=os.path.join(config["path"]["file_path"], "encoder_block"), format="jpeg")

### Create the Decoder

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, in_channels = 1024, out_channels = 512, is_last = False):
        super(DecoderBlock, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.is_last = is_last
        self.kernel = 2
        self.stride = 2
        self.padding = 0

        self.layers = OrderedDict()
        self.decoder = self.block()

    def block(self):
        self.layers["convTranspose"] = nn.ConvTranspose2d(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            kernel_size=self.kernel,
            stride=self.stride,
            padding=self.padding,
        )

        if self.is_last:
            self.layers["Tanh"] = nn.Tanh()
        else:
            self.layers["conv"] = nn.Conv2d(
                in_channels=self.out_channels,
                out_channels=self.out_channels,
                kernel_size=3,
                stride=1,
                padding=1,  # Changed to maintain spatial dimensions
            )
            self.layers["relu"] = nn.ReLU(inplace=True)

            self.layers["conv_1"] = nn.Conv2d(
                in_channels=self.out_channels,
                out_channels=self.out_channels,
                kernel_size=3,
                stride=1,
                padding=1,  # Changed to maintain spatial dimensions
            )
            self.layers["batch_norm"] = nn.BatchNorm2d(num_features=self.out_channels)
            self.layers["relu_1"] = nn.ReLU(inplace=True)

        return nn.Sequential(self.layers)

    def forward(self, x):
        if x is not None:
            return self.decoder(x)

        else:
            raise ValueError("The input must not be None.".capitalize())

    @staticmethod
    def total_params(model = None):
        if model is not None:
            return sum(params.numel() for params in model.parameters())

        else:
            raise ValueError("The model must not be None.".capitalize())

    @staticmethod
    def model_architecture(model=None):
        config = params()
        if model is not None:
            if os.path.exists(config["path"]["file_path"]):

                draw_graph(
                    model=model, input_size=(1, 3, 256, 256)
                ).visual_graph.render(
                    filename=os.path.join(config["path"]["file_path"], "decoder_block"),
                    format="jpg",
                )

            else:
                raise FileNotFoundError(
                    "The processed data folder does not exist. Please check the path and try again."
                )

        else:
            raise ValueError("The model must not be None.".capitalize())


if __name__ == "__main__":
    in_channels = 1024
    out_channels = 512
    num_repetitive = 5
    
    layers = []
    
    for idx in range(num_repetitive):
        layers.append(DecoderBlock(
            in_channels=in_channels,
            out_channels= 3 if idx == num_repetitive - 1 else out_channels,
            is_last= True if idx == num_repetitive - 1 else False))
        
        in_channels = out_channels
        out_channels //= 2
        
    model = nn.Sequential(*layers)
    
    print(model(torch.randn(1, 1024, 8, 8)).size())
    
    print(summary(model=model, input_size=(1024, 8, 8)))
    
    print(DecoderBlock.total_params(model=model))
    
    DecoderBlock.model_architecture(model=model)

#### Create the AutoEncoder

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, in_channels=3):
        super(AutoEncoder, self).__init__()

        self.in_channels = in_channels
        self.out_channels = 64
        self.num_repetitive = 5

        self.layers = []

        for _ in range(self.num_repetitive):
            self.layers.append(
                EncoderBlock(
                    in_channels=self.in_channels, out_channels=self.out_channels
                )
            )
            self.in_channels = self.out_channels
            self.out_channels *= 2

        self.out_channels = self.in_channels // 2

        for idx in range(self.num_repetitive):
            self.layers.append(
                DecoderBlock(
                    in_channels=self.in_channels,
                    out_channels=(
                        3 if idx == self.num_repetitive - 1 else self.out_channels
                    ),
                    is_last=True if idx == self.num_repetitive - 1 else False,
                )
            )

            self.in_channels = self.out_channels
            self.out_channels //= 2

        self.model = nn.Sequential(*self.layers)

    def forward(self, x):
        if x is not None:
            return self.model(x)

        else:
            raise ValueError("The input must not be None.".capitalize())

    @staticmethod
    def total_params(model=None):
        if model is not None:
            return sum(params.numel() for params in model.parameters())

        else:
            raise ValueError("The model must not be None.".capitalize())

    @staticmethod
    def model_architecture(model=None):
        config = params()
        if model is not None:
            if os.path.exists(config["path"]["file_path"]):

                draw_graph(
                    model=model, input_size=(1, 3, 256, 256)
                ).visual_graph.render(
                    filename=os.path.join(config["path"]["file_path"], "autoencoder"),
                    format="jpg",
                )

            else:
                raise FileNotFoundError(
                    "The processed data folder does not exist. Please check the path and try again."
                )

        else:
            raise ValueError("The model must not be None.".capitalize())

#### Helper code

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

def load_dataloader():
    config = params()

    if os.path.exists(config["path"]["processed_path"]):
        train_dataloader = load(
            filename=os.path.join(
                config["path"]["processed_path"], "train_dataloader.pth"
            )
        )
        test_dataloader = load(
            filename=os.path.join(
                config["path"]["processed_path"], "test_dataloader.pth"
            )
        )

        return {
            "train_dataloader": train_dataloader,
            "test_dataloader": test_dataloader,
        }

    else:
        raise Exception("Could not find the file".capitalize())


def helpers(**kwargs):
    lr = kwargs["lr"]
    adam = kwargs["adam"]
    SGD = kwargs["SGD"]
    lr_scheduler = kwargs["lr_scheduler"]

    config = params()

    model = AutoEncoder(in_channels=3)

    if adam:
        optimizer = optim.Adam(
            params=model.parameters(),
            lr=lr,
            betas=(config["model"]["beta1"], config["model"]["beta2"]),
        )

        if lr_scheduler:
            scheduler = StepLR(
                optimizer=optimizer,
                step_size=config["model"]["step_size"],
                gamma=config["model"]["gamma"],
            )

    elif SGD:
        optimizer = optim.SGD(
            params=model.parameters(),
            lr=lr,
            momentum=config["model"]["momentum"],
        )

        if lr_scheduler:
            scheduler = StepLR(
                optimizer=optimizer,
                step_size=config["model"]["step_size"],
                gamma=["model"]["gamma"],
            )

    dataloader = load_dataloader()

    return {
        "model": model,
        "optimizer": optimizer,
        "scheduler": scheduler,
        "train_dataloader": dataloader["train_dataloader"],
        "test_dataloader": dataloader["test_dataloader"],
    }


if __name__ == "__main__":
    helpers()